In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.00001
Capacitance = 1.


solver = "ca"
solver = "ca_canonical"
# solver = "ca_canonical_X"
# solver = "gp_canonical_X"
# solver = "qcqp_sni"
# solver = "cvx_dccp"

time_limit = 100

# Ciropt problem

In [3]:
problem = co.gradient_flow_circuit(mu, L_smooth, Capacitance)
problem.obj = problem.b
bounds = {  
            # 'b': {"ub": 10, "lb": 1},
            # 'd': {"ub": 10, "lb": 0.},
            'h': {"ub": 1, "lb": 1},
            'alpha': {"ub": 0.5, "lb": 0.5},
            'beta': {"ub": 0.5, "lb": 0.5},
            }

res, model, sp_exp = problem.solve(solver=solver, verbose=False, bounds=bounds, time_limit=100)[:3]
print(res)

dim_G=5, dim_F=4
{'b': 0.9330127549898344, 'h': 1.000000007180752, 'd': 0.0, 'alpha': 0.4999999929957276, 'beta': 0.5000000065841887}


# PEP verification

In [4]:
res

{'b': 0.9330127549898344,
 'h': 1.000000007180752,
 'd': 0.0,
 'alpha': 0.4999999929957276,
 'beta': 0.5000000065841887}

In [5]:
# {'b': 0.9999999972740821,
#  'h': 2.3476576020226845,
#  'd': 0.0,
#  'alpha': 0.42594920754454807,
#  'beta': 0.42592981378027417}


# {'alpha': 9.098402068796711e-05,
#  'b': 0.9999999898240947,
#  'beta': 0.5039539138113834,
#  'd': 9497.932149988383,
#  'h': 1.000108919758979}

In [6]:
problem = co.gradient_flow_circuit(mu, L_smooth, Capacitance, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=-2.690665634741407e-09
